In [5]:
from sqlalchemy import create_engine
from sshtunnel import SSHTunnelForwarder

In [6]:
creds = {
    'DB_HOST': 'liradb.compute.dtu.dk',
    'DB_PORT': 5435,
    'DB_USER': 'guest',
    'DB_PASS': 'V2GjxQVn',
    'DB_NAME': 'postgres',
    'SSH_HOST': 'thinlinc.compute.dtu.dk',
    'SSH_USER': 's174420',
    'SSH_PASS': open('creds.json').readline()
}

In [7]:
print('Connecting to the SSH Tunnel...')

ssh_tunnel = SSHTunnelForwarder(
    ssh_address_or_host=(creds['SSH_HOST'], 22),
    ssh_username=creds["SSH_USER"],
    ssh_password=creds['SSH_PASS'],
    remote_bind_address=(creds["DB_HOST"], creds['DB_PORT'])
)

ssh_tunnel.start()
print('Success')

Connecting to the SSH Tunnel...


In [8]:
print('Connecting to the PostgreSQL Database...')

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
    host='localhost',
    port=ssh_tunnel.local_bind_port,
    user=creds["DB_USER"],
    password=creds["DB_PASS"],
    db=creds["DB_NAME"]
))

engine.connect()

Connecting to the PostgreSQL Database...


In [11]:
with engine.connect() as conn:
    res = conn.execute('select * from "Devices" limit 100;')

    for row in res:
        print(row)

(UUID('8051a60b-84f8-4c7e-9e6a-cd57bcf52ebe'), datetime.datetime(2020, 4, 30, 9, 14, 6, 484286, tzinfo=datetime.timezone.utc), datetime.datetime(1, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), UUID('e9445538-3381-414c-a302-2b33b70d72b1'))
(UUID('9491599b-0e08-438c-a7cb-39b10f34b2de'), datetime.datetime(2020, 4, 30, 10, 23, 43, 280443, tzinfo=datetime.timezone.utc), datetime.datetime(1, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), UUID('e9445538-3381-414c-a302-2b33b70d72b1'))
(UUID('6513ab9d-66f0-4701-a2fb-a54dbaf12151'), datetime.datetime(2020, 4, 30, 10, 44, 15, 16908, tzinfo=datetime.timezone.utc), datetime.datetime(1, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), UUID('e9445538-3381-414c-a302-2b33b70d72b1'))
(UUID('bded9c90-025e-460a-9c30-b65a29d07b91'), datetime.datetime(2020, 4, 30, 22, 10, 33, 815965, tzinfo=datetime.timezone.utc), datetime.datetime(1, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), UUID('e9445538-3381-414c-a302-2b33b70d72b1'))
(UUID('eadc9490-e9de-4219-ba29-03472c74cefd'), 

In [ ]:
engine.connect()

In [ ]:
engine.